# 转置卷积

- **转置卷积不是卷积的逆运算**
    - 只是将特征图的大小，还原为卷积之前的特征图大小，但**数值与之前的特征图不同**
- **转置卷积也是一种卷积**
    - 他将输入和核进行了**重新排列**
    - 同卷积一般是做下采样不同，它通常用作**上采样**
    - 如果卷积将输入从$(h, w)$变成了$(h', w')$，同样超参数下它将$(h', w')$变成了$(h, w)$
- 卷积不会增大高宽：要么不变，要么变小
- 转置卷积的用处：
    - 在语义分割中，卷积不断减少高宽，如何做像素级别的输出。所以需要转置卷积把小的高宽变大
- 转置卷积增大输出图片的高宽
    - stride越大，输出的高宽也越大

    <img src="./pic/转置卷积作用.PNG" width=400 height=400>
- 计算：
<img src="./pic/转置卷积.PNG" width=400 height=400>
<img src="./pic/转置卷积计算.PNG" width=400 height=400>

## 转置卷积运算步骤

1. 在输入特征图元素间填充s-1行、列0
2. 在输入特征图四周填充k-p-1行、列0
3. 将卷积核参数上下、左右翻转
4. 做正常卷积运算（填充0，步距1）

<table><tr>
    <td><img src="./pic/转置卷积s=1, p=0, k=3.gif" width=200 height=200>s=1, p=0, k=3</td>
    <td><img src="./pic/转置卷积s=2, p=0, k=3.gif" width=200 height=200>s=1, p=0, k=3</td>
    <td><img src="./pic/转置卷积s=2, p=1, k=3.gif" width=200 height=200>s=1, p=0, k=3</td>
</tr></table>

- 公式：
    - $H_{out} = (H_{in} - 1) \times stride - 2 \times padding + kernel Size$
    - $W_{out} = (W_{in} - 1) \times stride - 2 \times padding + kernel Size$

## 转置卷积运算例子（变成卷积的输入与核，再做卷积运行）：
- **最后的卷积操作中（填充0，步幅1），与转置卷积中的填充与步幅无关**

- 例1：高宽 = $2 \times 2$, k = 2, p = 0, s = 1
    - 输出为：(2 - 1) x 1 - 2 x 0 + 2 = 3
<img src="./pic/转置卷积运算例子1.PNG" width=400 height=400>

- 例2：高宽 = $2 \times 2$, k = 2, p = 1, s = 1
    - 输出为：(2 - 1) x 1 - 2 x 1 + 2 = 1
<img src="./pic/转置卷积运算例子2.PNG" width=400 height=400>

- 例3：高宽 = $2 \times 2$, k = 2, p = 0, s = 2
    - 输出为：(2 - 1) x 2 - 2 x 0 + 2 = 4
<img src="./pic/转置卷积运算例子3.PNG" width=400 height=400>
<img src="./pic/转置卷积运算例子3手写.jpg" width=400 height=400>

## 形状换算
- 如果$(n-k-2p+s)$刚好可以整除$s$，则$n=sn'+k-2p-s$，否则大于
<img src="./pic/转置卷积形状换算1.PNG" width=400 height=400>

## 为什么叫转置

- 卷积：$Y'(n \times 1) = V(n \times m) \odot X'(m \times 1)$
- 转置：$Y'(m \times 1) = V(m \times n) \odot X'(n \times 1)$
- 这里的向量版本应该指将矩阵flatten成列向量
<img src="./pic/为什么叫转置.PNG" width=400 height=400>


## 同反卷积的关系
<img src="./pic/同反卷积的关系.PNG" width=400 height=400>

# 转置卷积的实现

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 实现基本的转置卷积运算

- 无padding，stride=1

In [2]:
def trans_conv(X, K):
    h, w = K.shape
    # 卷积高宽计算公式（图片h - 核h + 1）
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

## 验证上述实现输出

In [3]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

## 使用高级API获得相同的结果

In [4]:
# 1：批量大小
# 1：通道数
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
# 1：输入通道数
# 1：输出通道数
# 不需要偏差
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<SlowConvTranspose2DBackward>)

# 填充、步幅和多通道

- 卷积高宽计算公式：
    - 高：$(图h - 核h + padding + 步长) \div 步长$
    - 宽：$(图w - 核w + padding + 步长) \div 步长$
- 转置卷积高宽计算公式：
    - $H_{out} = (H_{in} - 1) \times stride - 2 \times padding + kernel Size$
    - $W_{out} = (W_{in} - 1) \times stride - 2 \times padding + kernel Size$
    
- 与常规卷积不同，在转置卷积中，填充被应用于的输出（常规卷积将填充应用于输入）
    - 例如，当将高和宽两侧的填充数指定为1时，转置卷积的输出中将删除第一和最后的行与列。
    - Padding是指上采样之后的图对应于原图是否扩充了
- 多通道：与卷积相同

In [6]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

# 原tconv(X)是一个3x3矩阵
# padding=1后
# 成单个元素（输出的大小变小了）
# 填充在输出上，本来2*2与2*2转置卷积得到3*3，填充在输出上，就是填充在3*3上，得到的结果就成了一个数值

tensor([[[[4.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [7]:
# （图高+核高-步长）* 步长=（2 + 2 - 2）* 2 = 4
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<SlowConvTranspose2DBackward>)

In [8]:
# 转置卷积主要是作用在单通道上面。对于多通道，与卷积相同

X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)    # w =（16-5+2*2+3）/3= 6 
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)    # w = (6 - 1)*3 - 2*2 + 5 = 16 
tconv(conv(X)).shape == X.shape

True

### 将特征图的大小，还原为卷积之前的特征图大小，但数值与之前的特征图不同

In [14]:
tconv(conv(X)).sum == X.sum

False

## 与矩阵变换的联系

In [17]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

- 给一个kernel，变成上文的V。变成一个大矩阵，使得卷积变成矩阵乘法

In [18]:
# 就是将卷积的运算变为了矩阵和向量的乘法，笨办法手推一下就好
def kernel2matrix(K):
    # 之所以W是4x9，是因为输入是3x3，拉长是长为9的向量（1行9列）。W*X，所以W的列数是9
    # 因为输出是2x2，所以要做4次卷积操作，所以有4行，每行对应输出元素的卷积
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W
# [1., 2., 0., 3., 4., 0., 0., 0., 0.]：卷积核k，对第1块

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [20]:
X.reshape(-1)

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])

In [19]:
# Y是用卷积做出来的Y
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [21]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [ ]:
转置而不是逆运算，所以形状是变回去了，但数值不会变回X